In [1]:
import sys
import warnings
import pathlib

import pandas as pd
from copairs import map
from pycytominer.cyto_utils import load_profiles
from pycytominer import annotate
from tqdm import TqdmWarning

sys.path.append("../../")
from src import io_utils, data_utils

# removing warnigns 
warnings.filterwarnings("ignore", category=TqdmWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

/home/erikserrano/Programs/miniconda3/envs/cfret-map/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This code sets up the necessary file paths and directories required for the notebook, ensuring that input files exist. 
It also creates a results folder if it doesn't already exist to store outputs generated during the analysis.

In [2]:
# Setting the base data directory and ensure it exists (raises an error if it doesn't)
data_dir = pathlib.Path("../data/").resolve(strict=True)

# Setting the metadata directory for updated plate maps and ensure it exists
metadata_dir = pathlib.Path("../data/metadata/updated_platemaps").resolve(strict=True)

# Path to the updated barcode plate map file, ensure it exists
platemap_path = (metadata_dir / "updated_barcode_platemap.csv").resolve(strict=True)

# Path to the configuration file (does not enforce existence check here)
config_path = pathlib.Path("../config.yaml").resolve(strict=True)

# Setting the results directory, resolve the full path, and create it if it doesn't already exist
results_dir = pathlib.Path("./results").resolve()
results_dir.mkdir(exist_ok=True)

Loading in the files

In [3]:
# loading config and general configs
configs = io_utils.load_config(config_path)
general_configs = configs["general_configs"]

# loading bar code
barcode = pd.read_csv(platemap_path)

Since these files have undergone feature selection, it is essential to identify the overlapping feature names to ensure accurate and consistent analysis.

In [4]:
shared_cols = None
for aggregated_profile in list(data_dir.glob("*.parquet")):
    # read aggreagated profiled and column names
    agg_df = pd.read_parquet(aggregated_profile)
    columns = list(agg_df.columns)

    # Update the shared_columns set
    if shared_cols is None:
        # Initialize shared columns with the first profile's columns, preserving order
        shared_cols = columns
    else:
        # Retain only the columns present in both the current profile and shared columns
        shared_cols = [col for col in shared_cols if col in columns]


In this section, the code processes and organizes data by grouping related files and enriching them with additional metadata. Each group is assigned a unique identifier, and the corresponding data files are systematically loaded and prepared. New metadata columns are generated by combining existing information to ensure consistency and clarity. Additional metadata is integrated into the data to provide valuable experimental context, while unique identifiers are added to distinguish the aggregated profiles from different batches.

In [5]:
# Suffix for aggregated profiles
aggregated_file_suffix = "aggregated.parquet"

# Dictionary to store loaded plate data grouped by batch
loaded_plate_batches = {}

# Iterate over unique platemap files and their associated plates
for batch_index, (platemap_filename, associated_plates_df) in enumerate(
    barcode.groupby("platemap_file")
):
    # Generate a unique batch ID
    batch_id = f"batch_{batch_index + 1}"

    # Load the platemap CSV file
    platemap_path = (metadata_dir / f"{platemap_filename}.csv").resolve(strict=True)
    platemap_data = pd.read_csv(platemap_path)

    # Extract all plate names associated with the current platemap
    plate_barcodes = associated_plates_df["plate_barcode"].tolist()

    # List to store all loaded and processed aggregated plates for the current batch
    loaded_aggregated_plates = []

    for plate_barcode in plate_barcodes:
        # Resolve the file path for the aggregated plate data
        plate_file_path = (
            data_dir / f"{plate_barcode}_{aggregated_file_suffix}"
        ).resolve(strict=True)

        # Load the aggregated profile data for the current plate
        aggregated_data = load_profiles(plate_file_path)

        # Create a new metadata column called 'Metadata_well' by concatenating well row and column
        # Zero-pad single-digit well column numbers for consistency (e.g., "B2" -> "B02")
        aggregated_data["Metadata_well"] = aggregated_data["Metadata_WellRow"].astype(
            str
        ) + aggregated_data["Metadata_WellCol"].astype(int).apply(lambda x: f"{x:02}")

        # Reorder columns to place 'Metadata_well' as the first column
        aggregated_data = aggregated_data[
            ["Metadata_well"]
            + [col for col in aggregated_data.columns if col != "Metadata_well"]
        ]

        # Annotate the aggregated profile with metadata from the platemap
        aggregated_data = annotate(
            aggregated_data,
            platemap_data,
            join_on=["Metadata_well_position", "Metadata_well"],
        )

        # split metadata features and morphology features
        meta, _ = data_utils.split_meta_and_features(profile=aggregated_data)

        # update aggregated_data with only shared features
        aggregated_data = aggregated_data[meta + shared_cols]

        # Add a new column indicating the source plate for each row
        aggregated_data.insert(0, "Metadata_plate_barcode", plate_barcode)

        # Append the processed aggregated data for this plate to the batch list
        loaded_aggregated_plates.append(aggregated_data)

    # Combine all processed plates for the current batch into a single DataFrame
    combined_aggregated_data = pd.concat(loaded_aggregated_plates)

    # Store the combined DataFrame in the loaded_plate_batches dictionary
    loaded_plate_batches[batch_id] = combined_aggregated_data


In this section, we analyze a high-content screening dataset generated from cell painting experiments, where failing cardiac fibroblasts are treated with multiple compounds. Our goal is to calculate the mean average precision (mAP) by comparing the experimental treatments to two controls: a negative control consisting of DMSO-treated failing cardiac fibroblasts and a positive control consisting of DMSO-treated healthy cardiac fibroblasts.

We start by preparing the dataset, copying the profiles, and assigning a reference index to ensure proper grouping of non-DMSO treatment replicates. Metadata and feature columns are separated to facilitate the calculation of average precision (AP) scores. To calculate these scores, we define positive pairs as treatments with the same metadata values (e.g., same treatment type) across all plates. Negative pairs, on the other hand, are determined by comparing all DMSO-treated wells across all plates with all other treatments.

Once the AP scores are computed, we aggregate them across all plates for each treatment to derive the mean average precision (mAP) score. This process captures the consistency of treatment performance relative to the controls and allows for a comprehensive evaluation of the dataset. Finally, we save both the AP and mAP scores for each control condition, providing a well-structured dataset for further interpretation and downstream analysis.

In [6]:
# Load configurations for average precision (AP) and mean average precision (mAP)
copairs_ap_configs = configs["copairs_ap_configs"]
copairs_map_configs = configs["copairs_map_configs"]

# Define control conditions for the analysis
# Each tuple specifies the control type, treatment, and associated cell state
control_list = [("negative", "DMSO", "failing"), ("positive", "DMSO", "healthy")]

# Iterate over each batch of loaded plate profiles
for batch_id, profile in loaded_plate_batches.items():
    # Analyze the profile for each control condition
    for control_type, control_treatment, cell_state in control_list:
        # Create a copy of the profile to preserve the original data
        profile = profile.copy()

        # Assign a default reference index based on the row index
        profile["Metadata_reference_index"] = profile.index

        # Mark all non-control replicates (e.g., treatments not matching the current control)
        profile.loc[
            (profile["Metadata_treatment"] != control_treatment)
            & (profile["Metadata_cell_type"] != cell_state),
            "Metadata_reference_index",
        ] = -1

        # Move the "Metadata_reference_index" column to the beginning for clarity
        profile.insert(
            0, "Metadata_reference_index", profile.pop("Metadata_reference_index")
        )

        # Separate metadata columns from feature columns for downstream calculations
        meta_columns, feature_columns = data_utils.split_meta_and_features(profile)

        # Calculate average precision (AP) for the profile
        # Positive pairs are based on treatments with the same metadata
        # Negative pairs compare DMSO-treated wells to all treatments
        replicate_aps = map.average_precision(
            meta=profile[meta_columns],
            feats=profile[feature_columns].values,
            pos_sameby=copairs_ap_configs["pos_sameby"],
            pos_diffby=copairs_ap_configs["pos_diffby"],
            neg_sameby=[],
            neg_diffby=copairs_ap_configs["neg_diffby"],
        )

        # Remove duplicate columns from the results, if any
        replicate_aps = replicate_aps.loc[:, ~replicate_aps.columns.duplicated()]

        # Exclude wells treated with the control treatment (DMSO)
        replicate_aps = replicate_aps.loc[
            replicate_aps["Metadata_treatment"] != control_treatment
        ]

        # Save the calculated AP scores to a file for further analysis
        replicate_aps.to_csv(
            results_dir / f"{control_type}_control_{cell_state}_{control_treatment}_AP_scores.csv",
            index=False,
        )

        # Calculate mean average precision (mAP) from the AP scores
        replicate_maps = map.mean_average_precision(
            replicate_aps,
            sameby=copairs_map_configs["same_by"],  # Grouping criteria for mAP
            null_size=copairs_map_configs["null_size"],  # Null distribution size
            threshold=copairs_map_configs["threshold"],  # Significance threshold
            seed=general_configs["seed"],  # Seed for reproducibility
        )

        # Save the mAP scores to a file for reporting
        replicate_maps.to_csv(
            results_dir / f"{control_type}_control_{cell_state}_{control_treatment}_mAP_scores.csv",
            index=False,
        )
